In [4]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dset
from torchvision.models import resnet18
import torch.optim as optim
import torchvision.transforms as transforms
import datetime
from torch.nn import init, Linear, ReLU, Softmax
from torch.nn.init import xavier_uniform_
from torch.nn.init import xavier_uniform_

image_size = (224,224)
transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
train_data=dset.CIFAR10(root='./Dataset',train=True,transform=transform,download=False)
test_data=dset.CIFAR10(root='./Dataset',train=False,transform=transform,download=False)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=20,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=20,shuffle=True)
train_len=len(train_data)
test_len=len(test_data)
print(train_len,test_len)

rs18=resnet18(pretrained=True, progress = True) #这里采用resnet18模型

#adjust resnet50 to my dataset
class r18(nn.Module):
    def __init__(self, model):
        super(r18,self).__init__()
        self.rn18 = model
        self.fl1 = nn.Linear(1000, 256)
        self.fl2 = nn.Linear(256,10)
        
    def forward(self, X):
        X = self.rn18(X)
        X = F.relu(self.fl1(X))
        X = F.dropout(X, p=0.25)
        X = self.fl2(X)
        return X
def get_num_correct(out, labels):  #求准确率
    return out.argmax(dim=1).eq(labels).sum().item()
model = r18(rs18)
criterion = nn.CrossEntropyLoss(reduction = "mean")
optim = torch.optim.SGD(model.parameters(),lr=0.01)

#put on cuda if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
nb_epochs = 15
accuracy = []
eps = 0.01
eps_iter = 0.001
steps = 20

for epoch in range(nb_epochs):
    running_loss = 0
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        model.eval()
        model.train()
        optim.zero_grad()
        prediction = model(images).to(torch.float32)
        loss = criterion(prediction, labels.to(torch.long)) 
        loss.backward()
        optim.step()

        running_loss += loss.item()
        
        #calculate accuracy
        corrects = 0
        pred = torch.argmax(prediction, dim = 1)
        
        for i, p,l in zip(range(64), pred, labels):
            corrects += (p == l)
        accuracy.append(int(corrects)/len(pred))
    
    print("Epoch: {j}/{total_epochs} \t Time: {time} \t Loss: {Loss} \t Accuracy: {acc}".format(j = epoch+1,Loss = running_loss/len(train_loader),total_epochs = nb_epochs,time = datetime.datetime.now().time(), acc=np.mean(accuracy[-50:])))  

50000 10000
cuda:0
Epoch: 1/15 	 Time: 01:24:21.374205 	 Loss: 0.39334962432533505 	 Accuracy: 0.9210000000000003
Epoch: 2/15 	 Time: 01:26:01.786989 	 Loss: 0.20061147493803874 	 Accuracy: 0.9390000000000001
Epoch: 3/15 	 Time: 01:27:40.370627 	 Loss: 0.14735993371370715 	 Accuracy: 0.9420000000000001
Epoch: 4/15 	 Time: 01:29:20.637380 	 Loss: 0.11121415296681225 	 Accuracy: 0.966
Epoch: 5/15 	 Time: 01:30:59.424599 	 Loss: 0.08254262488353416 	 Accuracy: 0.9610000000000001
Epoch: 6/15 	 Time: 01:32:39.890447 	 Loss: 0.06792509473658574 	 Accuracy: 0.9789999999999999
Epoch: 7/15 	 Time: 01:34:21.268450 	 Loss: 0.05823983196039917 	 Accuracy: 0.9770000000000002
Epoch: 8/15 	 Time: 01:36:02.793843 	 Loss: 0.05274443494292209 	 Accuracy: 0.981
Epoch: 9/15 	 Time: 01:37:44.304546 	 Loss: 0.03879450106025543 	 Accuracy: 0.987
Epoch: 10/15 	 Time: 01:39:22.546887 	 Loss: 0.037051780492003306 	 Accuracy: 0.9880000000000001
Epoch: 11/15 	 Time: 01:41:04.223595 	 Loss: 0.03652668577960212 	 A

In [5]:
# save model for our test notebook    
PATH = './cifar10_resnet18.pth'
torch.save(model.state_dict(), PATH)

In [7]:
model = r18(rs18)
model.load_state_dict(torch.load('./cifar10_resnet18.pth'))
correct_test = 0
model.cuda(0)
model.eval() 
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        test_features, test_labels = data
        outputs = model(test_features.cuda(0))
        _,pred_idxs = torch.topk(outputs, 1)
        correct_test += torch.eq(test_labels.cuda(0), pred_idxs.squeeze()).sum().item()
print(correct_test/len(test_data))

0.9504


In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dset
from torchvision.models import resnet50
import torch.optim as optim
import torchvision.transforms as transforms
import datetime
from torch.nn import init, Linear, ReLU, Softmax
from torch.nn.init import xavier_uniform_
from torch.nn.init import xavier_uniform_

image_size = (224,224)
transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
train_data=dset.CIFAR10(root='./Dataset',train=True,transform=transform,download=False)
test_data=dset.CIFAR10(root='./Dataset',train=False,transform=transform,download=False)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=20,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=20,shuffle=True)
train_len=len(train_data)
test_len=len(test_data)
print(train_len,test_len)

rs50=resnet50(pretrained=True, progress = True) #这里采用resnet18模型

#adjust resnet50 to my dataset
class r50(nn.Module):
    def __init__(self, model):
        super(r50,self).__init__()
        self.rn50 = model
        self.fl1 = nn.Linear(1000, 256)
        self.fl2 = nn.Linear(256,10)
        
    def forward(self, X):
        X = self.rn50(X)
        X = F.relu(self.fl1(X))
        X = F.dropout(X, p=0.25)
        X = self.fl2(X)
        return X
def get_num_correct(out, labels):  #求准确率
    return out.argmax(dim=1).eq(labels).sum().item()
model = r50(rs50)
criterion = nn.CrossEntropyLoss(reduction = "mean")
optim = torch.optim.SGD(model.parameters(),lr=0.01)

#put on cuda if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
nb_epochs = 5
accuracy = []
eps = 0.01
eps_iter = 0.001
steps = 20

for epoch in range(nb_epochs):
    running_loss = 0
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        model.eval()
        model.train()
        optim.zero_grad()
        prediction = model(images).to(torch.float32)
        loss = criterion(prediction, labels.to(torch.long)) 
        loss.backward()
        optim.step()

        running_loss += loss.item()
        
        #calculate accuracy
        corrects = 0
        pred = torch.argmax(prediction, dim = 1)
        
        for i, p,l in zip(range(64), pred, labels):
            corrects += (p == l)
        accuracy.append(int(corrects)/len(pred))
    
    print("Epoch: {j}/{total_epochs} \t Time: {time} \t Loss: {Loss} \t Accuracy: {acc}".format(j = epoch+1,Loss = running_loss/len(train_loader),total_epochs = nb_epochs,time = datetime.datetime.now().time(), acc=np.mean(accuracy[-50:])))  

50000 10000
cuda:0
Epoch: 1/5 	 Time: 02:57:48.885937 	 Loss: 0.3331942874856293 	 Accuracy: 0.9209999999999999
Epoch: 2/5 	 Time: 03:01:20.008677 	 Loss: 0.1535573620254174 	 Accuracy: 0.9470000000000002
Epoch: 3/5 	 Time: 03:04:39.292673 	 Loss: 0.10320970850944869 	 Accuracy: 0.9590000000000001
Epoch: 4/5 	 Time: 03:08:03.787511 	 Loss: 0.07606508990337607 	 Accuracy: 0.975
Epoch: 5/5 	 Time: 03:11:21.162412 	 Loss: 0.0590286113537848 	 Accuracy: 0.985


In [2]:
# save model for our test notebook    
PATH = './cifar10_resnet50.pth'
torch.save(model.state_dict(), PATH)

In [3]:
model = r50(rs50)
model.load_state_dict(torch.load('./cifar10_resnet50.pth'))
correct_test = 0
model.cuda(0)
model.eval() 
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        test_features, test_labels = data
        outputs = model(test_features.cuda(0))
        _,pred_idxs = torch.topk(outputs, 1)
        correct_test += torch.eq(test_labels.cuda(0), pred_idxs.squeeze()).sum().item()
print(correct_test/len(test_data))

0.9566


In [4]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dset
from torchvision.models import wide_resnet50_2
import torch.optim as optim
import torchvision.transforms as transforms
import datetime
from torch.nn import init, Linear, ReLU, Softmax
from torch.nn.init import xavier_uniform_
from torch.nn.init import xavier_uniform_

image_size = (224,224)
transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
train_data=dset.CIFAR10(root='./Dataset',train=True,transform=transform,download=False)
test_data=dset.CIFAR10(root='./Dataset',train=False,transform=transform,download=False)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=20,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=20,shuffle=True)
train_len=len(train_data)
test_len=len(test_data)
print(train_len,test_len)

wrs50=wide_resnet50_2(pretrained=True, progress = True) #这里采用resnet18模型

#adjust resnet50 to my dataset
class wr50(nn.Module):
    def __init__(self, model):
        super(wr50,self).__init__()
        self.wrn50 = model
        self.fl1 = nn.Linear(1000, 256)
        self.fl2 = nn.Linear(256,10)
        
    def forward(self, X):
        X = self.wrn50(X)
        X = F.relu(self.fl1(X))
        X = F.dropout(X, p=0.25)
        X = self.fl2(X)
        return X
def get_num_correct(out, labels):  #求准确率
    return out.argmax(dim=1).eq(labels).sum().item()
model = wr50(wrs50)
criterion = nn.CrossEntropyLoss(reduction = "mean")
optim = torch.optim.SGD(model.parameters(),lr=0.01)

#put on cuda if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
nb_epochs = 3
accuracy = []
eps = 0.01
eps_iter = 0.001
steps = 20

for epoch in range(nb_epochs):
    running_loss = 0
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        model.eval()
        model.train()
        optim.zero_grad()
        prediction = model(images).to(torch.float32)
        loss = criterion(prediction, labels.to(torch.long)) 
        loss.backward()
        optim.step()

        running_loss += loss.item()
        
        #calculate accuracy
        corrects = 0
        pred = torch.argmax(prediction, dim = 1)
        
        for i, p,l in zip(range(64), pred, labels):
            corrects += (p == l)
        accuracy.append(int(corrects)/len(pred))
    
    print("Epoch: {j}/{total_epochs} \t Time: {time} \t Loss: {Loss} \t Accuracy: {acc}".format(j = epoch+1,Loss = running_loss/len(train_loader),total_epochs = nb_epochs,time = datetime.datetime.now().time(), acc=np.mean(accuracy[-50:])))  

50000 10000
cuda:0
Epoch: 1/3 	 Time: 03:21:09.588077 	 Loss: 0.3373229045111686 	 Accuracy: 0.9359999999999999
Epoch: 2/3 	 Time: 03:26:21.064520 	 Loss: 0.13645767496023328 	 Accuracy: 0.961
Epoch: 3/3 	 Time: 03:31:25.862904 	 Loss: 0.0878499648787547 	 Accuracy: 0.9739999999999999


In [5]:
# save model for our test notebook    
PATH = './cifar10_wideresnet50.pth'
torch.save(model.state_dict(), PATH)

In [6]:
model = wr50(wrs50)
model.load_state_dict(torch.load('./cifar10_wideresnet50.pth'))
correct_test = 0
model.cuda(0)
model.eval() 
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        test_features, test_labels = data
        outputs = model(test_features.cuda(0))
        _,pred_idxs = torch.topk(outputs, 1)
        correct_test += torch.eq(test_labels.cuda(0), pred_idxs.squeeze()).sum().item()
print(correct_test/len(test_data))

0.9639
